### Ca Simulation

This notebook describes and implements a simulation of calcium imaging data for use to test source extraction methods with ground truth data.

## 1. Calculating kernel for GCaMP6s

In the CNMF framework, calcium transients are are generated by an AR(2) process. The appendex of [reference] shows that this model is a discrete-time approximation of a continuous model consisting of spikes convolved with a difference of exponential kernel. So to generate the simulation, the first step is to approximate the parameters of this kernel based on the measured response properties of GCaMP6s, given in [reference]. 

The kernel of the continuous process is given by 

$$h(t) = \exp \left(\frac{-t}{\tau_d} \right) - \exp \left(\frac{-t}{\tau_r} \right), $$

where $\tau_r$ is the rise time constant and $\tau_d$ is the decay time constant. [reference] lists values for the transient peak time, $t_{peak}$, and the half decay time $t_{half}$. To solve for $\tau_r$ and $\tau_d$ in terms of $\tau_{peak}$, and $\tau_{half}$, we will solve the follwing system of equations.

$$ h^{\prime} (t_{peak}) = 0$$

$$ h(t_{half}) = \frac{1}{2} h(t_{peak})$$

This system does not appear to have an analytic solution. However, by making a symplifying assumption, we can use the second equation to solve for $\tau_r$ in terms of $tau_d$. We assume that $t_{half} >> \tau_r$ so that $\exp \left( \frac{-t_{half}}{\tau_r} \right) \approx 0$. 

After solving for $\tau_r$ in terms of $tau_d$, we can substitute this the solution into the first equation and use numerical methods to find the solution for $\tau_d$. We can then substitute this back into the simplified second equation to find $\tau_r$. 

This is all done below using SymPy for the algebra and scipy optimize for the numerical methods.

In [66]:
import sympy as sp
from IPython.display import display, Math, Latex

t, tau_d, tau_r, tau_peak, tau_half = sp.symbols('t, tau_d, tau_r, tau_peak, tau_half')

def h(x):
    return sp.exp(-x/tau_d) - sp.exp(-x/tau_r)

eq1 = sp.Eq(sp.diff(h(t), t).subs(t, tau_peak), 0)

eq2 = sp.Eq(h(tau_half) - 0.5*h(tau_peak), 0)


solutions = sp.solve(eq1, tau_r)
assert(len(solutions)==1)

eq_sub = eq2.subs(tau_r, solutions[0])
tau_peak_val = 200
tau_half_val = 500
eq_sub = eq_sub.subs({tau_peak:tau_peak_val, tau_half:tau_half_val})
display(Math(sp.latex(eq_sub)))
f = sp.lambdify(tau_d, eq_sub.lhs, 'times')

times = np.arange(0,1000,1)
plt.plot(times, f(times))

<IPython.core.display.Math object>

NameError: 'times' module can't be used for lambdification

0.5*exp(-200*exp(-LambertW(-200*exp(-200/tau_d)/tau_d) - 200/tau_d)/tau_d) - exp(-500*exp(-LambertW(-200*exp(-200/tau_d)/tau_d) - 200/tau_d)/tau_d) - 0.5*exp(-200/tau_d) + exp(-500/tau_d)